## Reference

* [vit_jax_augreg.ipynb](https://colab.research.google.com/github/google-research/vision_transformer/blob/master/vit_jax_augreg.ipynb)

In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
# Load master table from Cloud.
with tf.io.gfile.GFile("gs://vit_models/augreg/index.csv") as f:
    df = pd.read_csv(f)

In [3]:
df.columns

Index(['name', 'ds', 'epochs', 'lr', 'aug', 'wd', 'do', 'sd', 'best_val',
       'final_val', 'final_test', 'adapt_ds', 'adapt_lr', 'adapt_steps',
       'adapt_resolution', 'adapt_final_val', 'adapt_final_test', 'params',
       'infer_samples_per_sec', 'filename', 'adapt_filename'],
      dtype='object')

In [4]:
# How many different pre-training datasets?
df["ds"].value_counts()

i21k       17238
i1k        17136
i21k_30    17135
Name: ds, dtype: int64

Filter based on the following criteria:

* Models should be pre-trained on ImageNet-21k and fine-tuned on ImageNet-1k.
* The final ImageNet-1k validation accuracy should be at least 75%. 

In [6]:
i21k_i1k_models = df.query("ds=='i21k' & adapt_ds=='imagenet2012'")
models_ge_75 = i21k_i1k_models.query("adapt_final_test >= 0.75")
models_ge_75.head()

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
1088,S/32,i21k,300.0,0.001,light0,0.03,0.0,0.0,0.433857,0.433740,...,imagenet2012,0.03,20000,384,0.836716,0.79670,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.03-do...,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.03-do...
1089,S/32,i21k,300.0,0.001,light0,0.03,0.0,0.0,0.433857,0.433740,...,imagenet2012,0.01,20000,384,0.831174,0.79138,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.03-do...,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.03-do...
1156,S/32,i21k,300.0,0.001,medium2,0.03,0.1,0.1,0.364121,0.363818,...,imagenet2012,0.03,20000,384,0.797612,0.76346,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...,S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-d...
1224,S/32,i21k,300.0,0.001,light0,0.10,0.0,0.0,0.434482,0.434307,...,imagenet2012,0.03,20000,384,0.830549,0.79482,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1-do_...,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1-do_...
1225,S/32,i21k,300.0,0.001,light0,0.10,0.0,0.0,0.434482,0.434307,...,imagenet2012,0.01,20000,384,0.832735,0.79680,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1-do_...,S_32-i21k-300ep-lr_0.001-aug_light0-wd_0.1-do_...


In [7]:
models_ge_75["name"].value_counts()

R26+S/32    112
B/32        109
S/16        108
R50+L/32    108
B/16        108
L/16        104
S/32         51
Ti/16        23
B/8           6
R+Ti/16       3
Name: name, dtype: int64

Now, we first fetch the maximum accuracies with respect to a given model type and then we pick the underlying models. 

In [8]:
best_scores_by_model_type = (
    models_ge_75.groupby("name")["adapt_final_test"].max().values
)
results = models_ge_75["adapt_final_test"].apply(
    lambda x: x in best_scores_by_model_type
)
models_ge_75[results].sort_values(by=["adapt_final_test"], ascending=False).head(10)

,name,ds,epochs,lr,aug,wd,do,sd,best_val,final_val,...,adapt_ds,adapt_lr,adapt_steps,adapt_resolution,adapt_final_val,adapt_final_test,params,infer_samples_per_sec,filename,adapt_filename
25977,L/16,i21k,300.0,0.001,medium1,0.10,0.1,0.1,0.512275,0.512275,...,imagenet2012,0.01,20000,384,0.913597,0.87118,304720000.0,49.87,L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...,L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...
26827,R50+L/32,i21k,300.0,0.001,medium2,0.10,0.0,0.0,0.506943,0.506602,...,imagenet2012,0.01,20000,384,0.909694,0.86210,111020000.0,326.73,R50_L_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....,R50_L_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....
24005,B/16,i21k,300.0,0.001,medium1,0.10,0.0,0.0,0.506260,0.506055,...,imagenet2012,0.01,20000,384,0.893771,0.86148,86860000.0,137.92,B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...,B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do...
50966,B/8,i21k,300.0,0.001,medium2,0.10,0.0,0.0,0.521426,0.521006,...,imagenet2012,0.01,20000,224,0.891742,0.85948,NaN,NaN,B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_...,B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_...
6390,R26+S/32,i21k,300.0,0.001,medium2,0.03,0.0,0.0,0.466338,0.466143,...,imagenet2012,0.03,20000,384,0.871371,0.84082,36470000.0,560.40,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....,R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0....
12376,S/16,i21k,300.0,0.001,light1,0.03,0.0,0.0,0.472676,0.472402,...,imagenet2012,0.03,20000,384,0.870824,0.83728,22200000.0,300.12,S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...,S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...
22576,B/32,i21k,300.0,0.001,medium1,0.03,0.0,0.0,0.473789,0.473525,...,imagenet2012,0.03,20000,384,0.878239,0.83620,88300000.0,954.94,B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-d...,B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-d...
1564,S/32,i21k,300.0,0.001,light1,0.03,0.0,0.0,0.432002,0.431943,...,imagenet2012,0.03,20000,384,0.830940,0.80012,22920000.0,2153.94,S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...,S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do...
32910,Ti/16,i21k,300.0,0.001,none,0.03,0.0,0.0,0.415176,0.414551,...,imagenet2012,0.03,20000,384,0.819934,0.78220,5790000.0,609.58,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...,Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_...
45081,R+Ti/16,i21k,300.0,0.001,none,0.03,0.0,0.0,0.390781,0.390459,...,imagenet2012,0.03,20000,384,0.793163,0.75402,6360000.0,2425.77,R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-d...,R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-d...


In [9]:
models_ge_75[results].sort_values(by=["adapt_final_test"], ascending=False).head(10)[
    "adapt_filename"
].values.tolist()

['L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_384',
 'R50_L_32-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_384',
 'B_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_384',
 'B_8-i21k-300ep-lr_0.001-aug_medium2-wd_0.1-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.01-res_224',
 'R26_S_32-i21k-300ep-lr_0.001-aug_medium2-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384',
 'S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384',
 'B_32-i21k-300ep-lr_0.001-aug_medium1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384',
 'S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384',
 'Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012-steps_20k-lr_0.03-res_384',
 'R_Ti_16-i21k-300ep-lr_0.001-aug_none-wd_0.03-do_0.0-sd_0.0--imagenet2012